In [2]:
#download illumina chip files
url = 'http://www.ebi.ac.uk/arrayexpress/files/A-GEOD-8882/A-GEOD-8882.adf.txt'
! wget --no-verbose --timestamping --directory-prefix download {url}
url = 'http://www.ebi.ac.uk/arrayexpress/files/A-GEOD-6434/A-GEOD-6434.adf.txt'
! wget --no-verbose --timestamping --directory-prefix download {url}

2015-08-17 08:17:56 URL:http://www.ebi.ac.uk/arrayexpress/files/A-GEOD-8882/A-GEOD-8882.adf.txt [24316605/24316605] -> "download/A-GEOD-8882.adf.txt" [1]
2015-08-17 08:18:15 URL:http://www.ebi.ac.uk/arrayexpress/files/A-GEOD-6434/A-GEOD-6434.adf.txt [24862681/24862681] -> "download/A-GEOD-6434.adf.txt" [1]


In [11]:
humanhap550 = set([])
hh550_fh = open("download/A-GEOD-6434.adf.txt")
for line in hh550_fh:
    if line.startswith('rs'):
        toks = line.strip().split('\t')
        humanhap550.add(toks[1])

print(str(len(humanhap550)) + " SNPs with rs#s on HH550")
o_fh = open("data/hh550-snp-for-bed.txt", 'w')
o_fh.write('\n'.join(humanhap550))
o_fh.close()

561303 SNPs with rs#s on HH550


In [12]:
humanomni1 = set([])
omni1_fh = open("download/A-GEOD-8882.adf.txt")
for line in omni1_fh:
    if line.startswith('rs'):
        toks = line.strip().split('\t')
        humanomni1.add(toks[1])

print(str(len(humanomni1)) + " SNPs with rs#s on HOmni1")
o_fh = open("data/ho1-snp-for-bed.txt", 'w')
o_fh.write('\n'.join(humanomni1))
o_fh.close()

970342 SNPs with rs#s on HOmni1


Follow instructions at http://www.gettinggeneticsdone.com/2011/06/mapping-snps-to-genes-for-gwas.html to get a BED file. Use GRCh37/hg19 and dbSNP Common SNPs(142). Name the downloaded files `bed-hh550.bed` and `bed-ho1.bed`. Move both of these files to `download/` and `gzip`.

In [1]:
#Download Entrez mappings.
url = 'http://files.figshare.com/230645/entrez.txt'
! wget --no-verbose --timestamping --directory-prefix download {url}

# add chr to the front to match the UCSC files
! sed 's/^/chr/' download/entrez.txt > download/chr_entrez.txt

2015-08-17 10:55:12 URL:http://files.figshare.com/230645/entrez.txt [473206/473206] -> "download/entrez.txt" [1]


In [1]:
# Count the overlaps
! windowBed -w 10000 -c -a download/chr_entrez.txt -b download/bed-hh550.bed.gz > data/hh550-entrez.bed
! windowBed -w 10000 -c -a download/chr_entrez.txt -b download/bed-ho1.bed.gz > data/ho1-entrez.bed